# Self-Driving Car Engineer Nanodegree


## Project 2: **Advanced Lane Finding** 
***
The goals / steps of this project are the following:

* Compute the camera calibration matrix and distortion coefficients given a set of chessboard images.
* Apply a distortion correction to raw images.
* Use color transforms, gradients, etc., to create a thresholded binary image.
* Apply a perspective transform to rectify binary image ("birds-eye view").
* Detect lane pixels and fit to find the lane boundary.
* Determine the curvature of the lane and vehicle position with respect to center.
* Warp the detected lane boundaries back onto the original image.
* Output visual display of the lane boundaries and numerical estimation of lane curvature and vehicle position.


## Import Packages

In [ ]:
#importing some useful packages
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import os
%matplotlib inline

## Read in a Sample Image

In [ ]:
#reading in an image
image = mpimg.imread('test_images/straight_lines1.jpg')

#printing out some stats and plotting
print('This image is:', type(image), 'with dimensions:', image.shape)
plt.imshow(image)  # if you wanted to show a single color channel image called 'gray', for example, call as plt.imshow(gray, cmap='gray')

## Helper Functions from Project 1

In [ ]:
import math

def grayscale(img):
    """
    Applies the Grayscale transform
    This will return an image with only one color channel
    but NOTE: to see the returned image as grayscale
    (assuming your grayscaled image is called 'gray')
    you should call plt.imshow(gray, cmap='gray')
    """
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    # Or use BGR2GRAY if you read an image with cv2.imread()
    # return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    masked_color = np.dstack((masked_image, masked_image, masked_image)) * 255
    
    return masked_image, masked_color


def weighted_img(img, initial_img, α=0.8, β=1., γ=0.):
    """
    `img` is the output of the hough_lines(), An image with lines drawn on it.
    Should be a blank image (all black) with lines drawn on it.
    
    `initial_img` should be the image before any processing.
    
    The result image is computed as follows:
    
    initial_img * α + img * β + γ
    NOTE: initial_img and img must be the same shape!
    """
    return cv2.addWeighted(initial_img, α, img, β, γ)

## Camera Calibration
### Extract Chessboard Corners

In [ ]:
import glob

# prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
nrows = 6
ncols = 9
objp = np.zeros((nrows*ncols,3), np.float32)
objp[:,:2] = np.mgrid[0:ncols, 0:nrows].T.reshape(-1,2)

# Arrays to store object points and image points from all the images.
objpoints = [] # 3d points in real world space
imgpoints = [] # 2d points in image plane.

# Make a list of calibration images
images = glob.glob('camera_cal/calibration*.jpg')

# Step through the list and search for chessboard corners
for idx, fname in enumerate(images):
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Find the chessboard corners
    ret, corners = cv2.findChessboardCorners(gray, (ncols,nrows), None)

    # If found, add object points, image points
    if ret == True:
        objpoints.append(objp)
        imgpoints.append(corners)

        # Draw and display the corners
        cv2.drawChessboardCorners(img, (ncols,nrows), corners, ret)
        #write_name = 'corners_found'+str(idx)+'.jpg'
        #cv2.imwrite(write_name, img)
        # 
        #Uncomment the two lines below to view the result
        #cv2.imshow('img', img)
        #cv2.waitKey(500)

cv2.destroyAllWindows()

### Compute Camera Calibration Matrix

In [ ]:
import pickle
%matplotlib inline

# Test undistortion on an image
img = cv2.imread('camera_cal/calibration1.jpg')
img_size = (img.shape[1], img.shape[0])

# Do camera calibration given object points and image points
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_size, None, None)

dst = cv2.undistort(img, mtx, dist, None, mtx)
cv2.imwrite('output_images/undistorted_chessboard.jpg',dst)

# Save the camera calibration result for later use
dist_pickle = {}
dist_pickle["mtx"] = mtx
dist_pickle["dist"] = dist
pickle.dump( dist_pickle, open( "camera_cal/wide_dist_pickle.p", "wb" ) )
#dst = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
# Visualize undistortion
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=30)
ax2.imshow(dst)
ax2.set_title('Undistorted Image', fontsize=30)

### Derivation of polynomial transformation from image coordinates to real coordinates

While most of the content in the helper functions is based on code and theory described in the lessons,
it might be useful to describe the logic used to tranform polynomials from the image coordinates to the
real coordinates since this was not explicitly described in the course lessons. This section therefore,
provides a brief description of the approach used to perform the tranformation.

Let the equation of the polynomial in the image coordinates be:

\begin{equation}
x_i = A_i y_i^{2} + B_i y_i + C_i
\end{equation}

In the above equation, $x$ is the dependent variable and $y$ is the independent
variable. Let the equation of the corresponding polynomial in the real coordinates be:

\begin{equation}
x_r = A_r y_r^{2} + B_r y_r + C_r
\end{equation}

Let $\lambda_x$ and $\lambda_y$ be the $x$ and $y$ distances (in meters) per pixel. The $x$ and $y$
coordinates can therefore be transformed as:

\begin{equation}
x_r = \lambda_{x} x_i  \\
y_r = \lambda_{y} x_i 
\end{equation}

If we substitute the above equations in to the polynomial expression in
real coordinates, we get:

\begin{equation}
x_i \lambda_x = A_i\lambda_y^2 y_i^{2} + B_i \lambda_y y_i + C_i \\
x_i = \frac{A_i\lambda_y^{2}}{\lambda_x} y_i^{2} + \frac{B_i \lambda_y}{\lambda_x} y_i + \frac{C_i}{\lambda_x}
\end{equation}

Comparing the above equation with the equation for the image coordinates and equating the coefficients, we get:

\begin{equation}
A_r = \frac{A_i\lambda_y^{2}}{\lambda_x} \\
B_r = \frac{B_i \lambda_y}{\lambda_x} \\
C_r = \frac{C_i}{\lambda_x}
\end{equation}

### Helper Functions for Project 2

In [ ]:
def edge_thresholds(img):
    # Extract R-channel
    R = img[:,:,0]

    # Extract S-channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    
    # Grayscale image
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sobel x
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))

    # Threshold x gradient
    thresh_min = 50
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    # Threshold R channel
    thresh_R = (150, 255)
    binary_R = np.zeros_like(R)
    binary_R[(R > thresh_R[0]) & (R <= thresh_R[1])] = 1
    
    # Threshold S channel
    s_thresh_min = 100
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1

    # Combine multiple binary images
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[((s_binary == 1) & binary_R == 1) | (sxbinary == 1) ] = 1
    #combined_binary[((s_binary == 1) & binary_R == 1)] = 1
    #combined_binary[((s_binary == 1)) | (sxbinary == 1) ] = 1

    color_image = np.dstack((combined_binary, combined_binary, combined_binary)) * 255
    return binary_R, s_binary, sxbinary, combined_binary, color_image

def topview(image):
    """
    Perform perspective transformation to obtain a top view of the image
    """
    
    src = np.float32([[600, 444], [675, 444], [1041, 676], [268, 676]])
    offsetv = 0
    offseth = 300
    img_size = (image.shape[1], image.shape[0])
    dst = np.float32([[offseth, offsetv], [img_size[0]-offseth, offsetv], 
                                     [img_size[0]-offseth, img_size[1]-offsetv], 
                                     [offseth, img_size[1]-offsetv]])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(image, M, image.shape[1::-1], flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv

def fit_poly(img_shape, leftx, lefty, rightx, righty):

    # Fit a second order polynomial to each with np.polyfit() ###
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    
    #Calc both polynomials using ploty, left_fit and right_fit ###
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    middle_fitx = 0.5*(left_fitx + right_fitx)
    middle_fit = np.polyfit(middle_fitx, ploty, 2)
    
    return left_fitx, right_fitx, middle_fitx, ploty, left_fit, right_fit, middle_fit

def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    
    # Create an output image to draw on and visualize the result
    stacked_windows_image = np.dstack((binary_warped, binary_warped, binary_warped))
    
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50

    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    # Step through the windows one by one
    for window in range(nwindows):
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height

        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        # Draw the windows on the visualization image
        cv2.rectangle(stacked_windows_image,(win_xleft_low,win_y_low),
                              (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(stacked_windows_image,(win_xright_low,win_y_low),
                              (win_xright_high,win_y_high),(0,255,0), 2) 
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xleft_low) &  (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
        (nonzerox >= win_xright_low) &  (nonzerox < win_xright_high)).nonzero()[0]

        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        #If number of pixes in windows exceeds minpix pixels, recenter next window
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass
    
    stacked_windows_image[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    stacked_windows_image[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]
    
    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, stacked_windows_image

def search_around_poly(binary_warped):
    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    # Extract the lane points
    leftx, lefty, rightx, righty, left_lane_inds, right_lane_inds, stacked_windows_image = find_lane_pixels(binary_warped)
        
    # Fit new polynomials
    left_fitx, right_fitx, middle_fitx, ploty, left_fit, right_fit, middle_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    # Create an image to draw on and an image to show the selection window
    binary_warped_color = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(binary_warped_color)
    
    # Identify the region between the left and right lanes
    left_region_limits = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    right_region_limits = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    region_pts = np.hstack((left_region_limits, right_region_limits))

    # Draw the polynomial fit for middle lane line
    mid_lower_limits = np.array([np.transpose(np.vstack([middle_fitx-5, ploty]))])
    mid_upper_limits = np.array([np.flipud(np.transpose(np.vstack([middle_fitx+5, ploty])))])
    mid_region_pts = np.hstack((mid_lower_limits, mid_upper_limits))

    # Draw the polynomial fit for left lane line
    left_lower_limits = np.array([np.transpose(np.vstack([left_fitx-5, ploty]))])
    left_upper_limits = np.array([np.flipud(np.transpose(np.vstack([left_fitx+5, ploty])))])
    left_region_pts = np.hstack((left_lower_limits, left_upper_limits))
    
    # Draw the polynomial fit for right lane line
    right_lower_limits = np.array([np.transpose(np.vstack([right_fitx-5, ploty]))])
    right_upper_limits = np.array([np.flipud(np.transpose(np.vstack([right_fitx+5, ploty])))])
    right_region_pts = np.hstack((right_lower_limits, right_upper_limits))
    
    # Write left, middle and right polynomial fits to image using different colors
    cv2.fillPoly(stacked_windows_image, np.int_([mid_region_pts]), (255,255, 0)) #yellow
    cv2.fillPoly(stacked_windows_image, np.int_([left_region_pts]), (255, 105, 180)) #pink
    cv2.fillPoly(stacked_windows_image, np.int_([right_region_pts]), (128, 0, 128)) #purple 
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([region_pts]), (0,255, 0))
    result = cv2.addWeighted(binary_warped_color, 1, window_img, 0.3, 0)
    
    # Draw the lane onto the warped blank image  
    result[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    result[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    return result, stacked_windows_image, left_fit, right_fit, middle_fit, \
                                          left_fitx, right_fitx, middle_fitx, ploty

def convert_to_real(poly_fit, xm_per_pix, ym_per_pix, poly_degree):
    """
    Convert from image coordinates to real coordinates using scaling factors
    """
    poly_fit_real = [0.0] * (poly_degree+1)

    for d in range(poly_degree+1):
        poly_fit_real[d] = poly_fit[d] * xm_per_pix / (ym_per_pix ** (poly_degree - d))

        return poly_fit_real

def measure_curvature(poly_fit, y_eval):
    '''
    Calculates the curvature of polynomial given the polynomial coefficients
    '''
    poly_curverad = ((1.0 + (2.0*poly_fit[0]*y_eval + poly_fit[1])**2)**1.5)/(2.0*poly_fit[0])
    
    return poly_curverad

def process_single_image(image, write_to_file = False):
    alpha = 1
    beta = 1
    gamma = 0
    
    # Specify corners of the quadrilateral masked region of interest
    imshape = image.shape
    mask_vertices = np.array([[(0.05*imshape[1],imshape[0]),
                      (0.47*imshape[1], 0.6*imshape[0]), 
                      (0.53*imshape[1], 0.6*imshape[0]), 
                      (0.95*imshape[1], imshape[0])]], dtype=np.int32)
    
    # Pixel to real world conversion factors
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/700 # meters per pixel in x dimension    

    # Write the original image to disk for reference
    if write_to_file:
        cv2.imwrite('output_images/original_image.png', cv2.cvtColor(image, cv2.COLOR_RGB2BGR))
 
    # Undistort the original image by applying camera calibrtion matrix
    undst = cv2.undistort(image, mtx, dist, None, mtx) 
    if write_to_file:
        cv2.imwrite('output_images/undistorted.png', cv2.cvtColor(undst, cv2.COLOR_RGB2BGR))
    
    # Use a combination of thresholds and color gradients to obtain an image
    # that retains the lane edges while eliminating irrelevant edges as much as possible
    binary_R, s_binary, sxbinary, edges_binary, edges_color = edge_thresholds(undst)
    if write_to_file:
        cv2.imwrite('output_images/edges.png', edges_color)
        R_color = np.dstack((binary_R, binary_R, binary_R))*255
        s_color = np.dstack((s_binary, s_binary, s_binary))*255
        sx_color = np.dstack((sxbinary, sxbinary, sxbinary))*255
        cv2.imwrite('output_images/R.jpg', R_color)
        cv2.imwrite('output_images/s.jpg', s_color)
        cv2.imwrite('output_images/sx.jpg', sx_color)

    # Remove all edges outside the region of interest defined by the vertices
    masked_image, masked_color = region_of_interest(edges_binary, mask_vertices)    
    if write_to_file:
        cv2.imwrite('output_images/masked_image.png', masked_color)
    
    # Transform the masked image to obtain a top view of the image through perspective transform
    topview_image, M, Minv = topview(masked_image)
    topview_color = np.dstack((topview_image, topview_image, topview_image)) * 255
    if write_to_file:
        cv2.imwrite('output_images/topview.png', topview_color)
    
    # Search the top view image to identify the lane region and fit polynomials to
    # the left, right and middle of the lane
    topview_region, stacked_windows_image, left_fit, right_fit, middle_fit, \
            left_fitx, right_fitx, middle_fitx, ploty = search_around_poly(topview_image)
    if write_to_file:
        cv2.imwrite('output_images/topview_region.png', topview_region) 
        cv2.imwrite('output_images/windowed_image.png', cv2.cvtColor(stacked_windows_image, cv2.COLOR_RGB2BGR))    

    # Convert polynomial coefficients from image to real coordinate system
    left_fit_real = convert_to_real(left_fit, xm_per_pix, ym_per_pix, 2)
    right_fit_real = convert_to_real(right_fit, xm_per_pix, ym_per_pix, 2)
    middle_fit_real = convert_to_real(middle_fit, xm_per_pix, ym_per_pix, 2)
    
    # Compute curvature for left, right and middle lines
    left_curverad = measure_curvature(left_fit_real, (image.shape[0])*ym_per_pix)
    right_curverad = measure_curvature(right_fit_real, (image.shape[0])*ym_per_pix)    
    middle_curverad = measure_curvature(middle_fit_real, (image.shape[0])*ym_per_pix)    
    
    if False:
        print('Left lane curvature (m) = ', left_curverad)
        print('Right lane curvature (m) = ', right_curverad)
        print('Middle lane curvature (m) = ', middle_curverad)
    
    # Transform the warped image back into the original image
    warped_back = cv2.warpPerspective(topview_region, Minv, topview_region.shape[1::-1], flags=cv2.INTER_LINEAR)
    if write_to_file:
        cv2.imwrite('output_images/lane_region.png', cv2.cvtColor(warped_back, cv2.COLOR_RGB2BGR))    
    
    # Compute mid point of the lane
    mid_lane = np.array([[middle_fitx[-1], ploty[-1]]], dtype = "float32")
    mid_lane = np.array([mid_lane])
    midpoint_lane = cv2.perspectiveTransform(mid_lane, Minv)
    
    # Compute midpoint of the image (assume this is the center of the car)
    mid_image = np.array([[image.shape[1]//2, ploty[-1]]], dtype = "float32")
    mid_image = np.array([mid_image])
    midpoint_image = cv2.perspectiveTransform(mid_image, Minv)
    offset = (midpoint_lane[0][0][0] - midpoint_image[0][0][0])*xm_per_pix

    # Combined the lane region markings to the original image
    result = weighted_img(warped_back, image, alpha, 1, gamma)
    if write_to_file:
        cv2.imwrite('output_images/weighted_image.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
        
    # Write radius of curvature and offset onto the image
    font = cv2.FONT_HERSHEY_SIMPLEX
    radius_loc = (int(0.1*image.shape[0]),int(0.05*image.shape[1]))
    offset_loc = (int(0.1*image.shape[0]),int(0.1*image.shape[1]))
    fontScale = 1.5
    fontColor = (255,255,255)
    lineType = 2
    radius_text = 'Radius of curvature = ' + str(middle_curverad) + '(m)'
    if offset > 0:
        offset_text = 'Vehicle is {0:0.2f}m right of center'.format(offset)
    else:
        offset_text = 'Vehicle is {0:0.2f}m left of center'.format(-offset)

    cv2.putText(result,radius_text, radius_loc, font, fontScale, fontColor, lineType)
    cv2.putText(result, offset_text, offset_loc, font, fontScale, fontColor, lineType)    
    if write_to_file:
        cv2.imwrite('output_images/annotated_image.png', cv2.cvtColor(result, cv2.COLOR_RGB2BGR))    
    
    return result    

### Test Sample Image

In [ ]:
#for imgpath in os.listdir("test_images/"):
for imgpath in ["straight_lines1.jpg"]:
    fullpath = os.path.join("test_images", imgpath)
    basename, ext = os.path.splitext(imgpath)
    imgpath_output = basename + '_output' + ext
    fullpath_output = os.path.join("output_images", imgpath_output)
    
    image = mpimg.imread(fullpath)
    result = process_single_image(image, write_to_file = True)
    
    cv2.imwrite(fullpath_output, cv2.cvtColor(result, cv2.COLOR_RGB2BGR))
    plt.imshow(result)

### Create Video

In [ ]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

In [ ]:
# Process video by proecessing the individual images that constitute the video
video_input = 'project_video.mp4'
video_output = 'output_images/project_video_output.mp4'
clip1 = VideoFileClip(video_input).subclip(0, None)
project_clip = clip1.fl_image(process_single_image) #NOTE: this function expects color images!!
%time project_clip.write_videofile(video_output, audio=False)

In [ ]:
# Embed video
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(video_output))